In [2]:
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
(x_train_inp , y_train_inp ), (x_test_inp , y_test_inp)  = mnist.load_data()
x_train_inp = x_train_inp/255.0
converted_ans = []

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [4]:
for i in range(len(y_train_inp)):
        ans = np.zeros(10)
        ans[y_train_inp[i]] = 1
        converted_ans.append(ans)

In [5]:
x_train , x_test , y_train , y_test = train_test_split(x_train_inp,converted_ans,test_size = 0.35,shuffle  = True)

In [33]:
class DigitModel:
    total_layers = []
    output_layers = []
    learning_rate = 0.0
    def __init__(self,lr):
        self.total_layers = []
        self.learning_rate = lr
    
    def add_layer(self,size_of_input_layer,number_of_neurons):
        weights_layer = np.random.randn(size_of_input_layer , number_of_neurons) 
        weights_layer = np.transpose(weights_layer)
        self.total_layers.append(weights_layer)
        
    def activation_sigmoid(self,x):
        return 1/(1 + np.exp(-x))

    def sigmoid_der(self,x):
        return (1 - x ) * x

    def loss_function(self,real, predict):
        sum = np.sum((real - predict) ** 2)
        return sum

    def predict_final(self,final_out):
        max = -1000
        index = -1
        for i in range(0,10):
            if (final_out[i] > max):
                max = final_out[i]
                index = i
        return index

    def predict_output(self,input_arr):
        prev = input_arr
        for layer in self.total_layers:
            prev = self.activation_sigmoid(np.dot(layer,prev))
            self.output_layers.append(prev)
        if (self.output_layers[-1].sum() != 0.0 and self.output_layers[-1].min() != 0 ):
            self.output_layers[-1] = self.output_layers[-1]/self.output_layers[-1].min()
            self.output_layers[-1] = self.output_layers[-1]/self.output_layers[-1].sum() 
        # hl_output = activation_sigmoid(np.dot(weights_ip,input_arr))
        # hl_output2 = activation_sigmoid(np.dot(weights_hl,hl_output))
        # final_output = activation_sigmoid(np.dot(weights_op,hl_output2))
        # if(final_output.sum() != 0.0 and final_output.min() != 0):
        #     final_output = final_output/final_output.min()
        #     final_output = final_output/final_output.sum()   
        # return output_layers[-1] 
    def backpropagation(self,input_data ,expected_output):
        self.output_layers.insert(0,input_data)
        # change_op = 2 * (output_layers[-1] - expected_output) * sigmoid_der(output_layers[-1])  # row wise multiply by *
        # weight_op_update = learning_rate * np.outer(change_op, hl_output2)    # every number multiplied by every other number in other array
        # total_layers[-1] -= weight_op_update
        prev = []
        for i in range(len(self.output_layers)-1,0,-1):
             if len(self.output_layers)-1 == i:
                change = 2*(self.output_layers[i]-expected_output) * self.sigmoid_der(self.output_layers[i])
                weight_op_update = self.learning_rate * np.outer(change, self.output_layers[i-1])
                self.total_layers[i-1] -= weight_op_update
             else:
                 change = np.dot(prev,self.total_layers[i])*self.sigmoid_der(self.output_layers[i])
                 weight_hl_update = self.learning_rate * np.outer(change,self.output_layers[i-1])
                 # print(i)
                 self.total_layers[i-1] -= weight_hl_update
             prev = change
        # change_hl = np.dot(change_op, weights_op) * sigmoid_der(hl_output2)
        # weight_hl_update = learning_rate * np.outer(change_hl,hl_output)
        # weights_hl -= weight_hl_update
        # change_ip = np.dot(change_hl, weights_hl) * sigmoid_der(hl_output)
        # weight_ip_update = learning_rate * np.outer(change_ip, input_data)
        # weights_ip -= weight_ip_update
    def fit(self,epochs):
        print(self.learning_rate)
        count = 0
        total = 0
        for i in range(epochs):
            ec = 0
            et = 0
            loss_sum = 0
            for j in range(len(x_train)):
                input_arr = np.reshape(x_train[j],784)
                self.predict_output(input_arr)
                loss_sum += self.loss_function(self.output_layers[-1],y_train[j])
                if(self.predict_final(self.output_layers[-1]) == self.predict_final(y_train[j])):
                    count = count + 1
                    ec = ec + 1
                self.backpropagation(input_arr,y_train[j])
                self.output_layers = []
                total = total + 1
                et = et + 1
            print("Epoch Loss : ",loss_sum/len(x_train))
            print("Epoch Accuracy : ", (ec/et)*100.0)
        accuracy = (count / total) * 100.0 
        print("Final accuracy : ",accuracy)
    def evaluate(self):
        count = 0
        loss_sum = 0
        total = 0
        for j in range(len(x_test)):
                input_arr = np.reshape(x_test[j],784)
                predict_output(input_arr)
                loss_sum += loss_function(self.output_layers[-1],y_test[j])
                if(self.predict_final(self.output_layers[-1]) == self.predict_final(y_test[j])):
                    count = count + 1
                self.output_layers = []
                total = total + 1
        print("Loss : ",loss_sum/len(x_test))
        accuracy = (count / total) * 100.0 
        print("Accuracy : ",accuracy)

In [34]:
digits = DigitModel(0.01)

digits.add_layer(784,100)
digits.add_layer(100,100)
digits.add_layer(100,10)


In [35]:
digits.fit(50)

Loss :  0.9571860898284758
Loss :  0.9571860898284758
Loss :  0.9571860898284758
Loss :  0.9571860898284758
Loss :  0.9571860898284758
Loss :  0.9571860898284758
Loss :  0.9571860898284758


KeyboardInterrupt: 

In [ ]:
digits.evaluate()